In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import random

The history saving thread hit an unexpected error (OperationalError('no such table: history')).History will not be written to the database.


In [2]:
x = torch.rand([1,2,2])
y = torch.rand([3,3,3]) 
print(x)
print(y)

tensor([[[0.3734, 0.4398],
         [0.0757, 0.1474]]])
tensor([[[0.7399, 0.9163, 0.0459],
         [0.2964, 0.3350, 0.5451],
         [0.4860, 0.3548, 0.1676]],

        [[0.9767, 0.4570, 0.1339],
         [0.5771, 0.8491, 0.5099],
         [0.6372, 0.5387, 0.0905]],

        [[0.4654, 0.4450, 0.8743],
         [0.5344, 0.5887, 0.4914],
         [0.8751, 0.2260, 0.5321]]])


In [4]:
class Module_conv_bn(nn.Module):
    
    def __init__(self) -> None:
        super(Module_conv_bn, self).__init__()
        self.conv = nn.Conv2d(1, 3, kernel_size=2, padding=1, stride=1, bias=True)
        self.batch = nn.BatchNorm1d(3)
    
    def forward(self, x):
        x = self.conv(x)
        #
        mean_x = x.sum()/sum(x.size())
        std = torch.sqrt(x.var())
        print(mean_x, std)
        #
        x = self.batch(x)

        return x, self.conv.weight.data, self.conv.bias.data, self.batch.weight.data, mean_x, std

In [15]:
learning_rate = 0.01

def loss(y_pred, y):
    return torch.sum(y - y_pred)

#loss = nn.MSELoss


In [30]:
mod = Module_conv_bn()
optimizer = torch.optim.SGD(mod.parameters(), lr = learning_rate)
num_epoch = 3
for epoch in range(num_epoch):
    Y_pred, wt_conv, bs_conv, wt_batch, mean_x, std = mod(x)
    Loss = loss(Y_pred, y)

    Loss.backward()

    optimizer.step()
    optimizer.zero_grad()

resn, wt_conv, bs_conv, wt_batch, mean_x, std = mod(x)
print(resn)

tensor(0.6302, grad_fn=<DivBackward0>) tensor(0.2824, grad_fn=<SqrtBackward0>)
tensor(0.6302, grad_fn=<DivBackward0>) tensor(0.2824, grad_fn=<SqrtBackward0>)
tensor(0.6302, grad_fn=<DivBackward0>) tensor(0.2824, grad_fn=<SqrtBackward0>)
tensor(0.6302, grad_fn=<DivBackward0>) tensor(0.2824, grad_fn=<SqrtBackward0>)
tensor([[[-1.1260, -0.3253, -1.3835],
         [-0.6737, -0.9837, -1.6349],
         [-1.0104, -1.0568, -1.3146]],

        [[ 0.2607,  0.7623,  1.7346],
         [ 0.9151,  1.0591,  0.7640],
         [ 0.8030,  0.9105,  0.7357]],

        [[ 0.6670,  0.3864,  1.4537],
         [ 0.8352,  0.9984,  1.1505],
         [ 1.0658,  1.1358,  1.1610]]], grad_fn=<NativeBatchNormBackward0>)


In [24]:
class Only_Conv(nn.Module):
    def __init__(self) -> None:
        super(Only_Conv, self).__init__()
        self.new_conv = nn.Conv2d(1, 3, kernel_size=2, padding=1, stride=1, bias=True)
        # I need to update date for that conv
        # w=
        self.new_conv.weight.data = wt_conv
        # bias = 
        self.new_conv.bias.data = bs_conv
    def forward(self, x):
        x = self.new_conv(x)
        x = self.batch(x)
        return x

In [31]:
new_mod = Only_Conv()
res = new_mod(x)
print(res.tolist())

[[[-1.1260391473770142, -0.32527828216552734, -1.383480429649353], [-0.6737300157546997, -0.9836843609809875, -1.634893774986267], [-1.0104479789733887, -1.05678391456604, -1.3146300315856934]], [[0.2607495188713074, 0.7623230218887329, 1.7345609664916992], [0.9150708913803101, 1.0590776205062866, 0.7640436887741089], [0.8029901385307312, 0.9105324149131775, 0.7357233166694641]], [[0.6669796705245972, 0.386446088552475, 1.4537380933761597], [0.8351789712905884, 0.9983914494514465, 1.150545358657837], [1.065812349319458, 1.1357749700546265, 1.1610283851623535]]]
